## Construct QIR for a QAOA circuit defined using OpenQAOA
    Generate QIR instructions using PyQIR

### Create a problem and generate instructions in QIR using OpenQAOA

In [ ]:
from openqaoa.problems.problem import MaximumCut
from openqaoa.workflows.optimizer import QAOA
from openqaoa.devices import create_device
from openqaoa.backends import QAOAQIR

In [ ]:
prob = MaximumCut.random_instance(n_nodes=7, edge_probability=0.5)
prob_qubo = prob.get_qubo_problem()

In [ ]:
q = QAOA()
q.set_circuit_properties(mixer_hamiltonian='x')
dev = create_device('local','qiskit.qasm_simulator')
q.set_device(dev)
q.compile(prob_qubo)

In [ ]:
circuit_params = q.backend.circuit_params
variate_params = q.variate_params

In [ ]:
qir_qaoa = QAOAQIR(circuit_params)
ir = qir_qaoa.convert_to_ir(variate_params)
print(ir)

In [ ]:
from pyqir.generator import ir_to_bitcode

In [ ]:
qaoa_bitcode = ir_to_bitcode(ir)
qaoa_bitcode

In [ ]:
# from azure.quantum.target.rigetti import RigettiTarget, Rigetti, Result, InputParams

In [ ]:
from azure.quantum import Workspace
workspace = Workspace(
    subscription_id = "",
    resource_group = "",
    name = "",
    location = "westus"
)

In [ ]:
# target = Rigetti(
#     workspace=workspace,
#     name=RigettiTarget.QVM,  # Defaults to RigettiTarget.QVM for simulation
# )

In [ ]:
import azure.quantum as azure_q
job = azure_q.Job.from_input_data(
    workspace=workspace,
    name="oq_test_with_qir",
    provider_id="rigetti",
    target="rigetti.sim.qvm",
    input_data_format="qir.v1",
    output_data_format="microsoft.quantum-results.v1",
    input_data=qaoa_bitcode,
    input_params={
        "shots": 100,
        "entryPoint": "main",
        "arguments": []
    }
)

In [ ]:
info = job.details.as_dict()
info

In [ ]:
job.get_results()